In [1]:
%pip install -q -U langchain torch transformers sentence-transformers datasets faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import pdfplumber
from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from transformers import AutoTokenizer, pipeline
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
pdf_path = "./p4.pdf"

model_id = "google/gemma-2-2b-it"
device = torch.device("mps" if torch.mps.is_available() else "cpu")
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {"device": "mps" if torch.mps.is_available() else "cpu"}
encode_kwargs = {"normalize_embeddings": False}

questions = [
        "List of Insured Members?",
        "What is the Room Rent included in the policy?",
        "What is the Maternity Sum capping or sum insured?",
        "What is the policy start date?",
        "Sum insured for the policy?",
        "Does the policy have copay?",
        "what is the policy inception date",
        "what is the waiting period? and list down the categories for waiting periods",
        "what is the waiting period for specific disease waiting periods",
        "what is the waiting period for maternity package ",
    ]

In [17]:
def extract_text(pdf_path, max_pages=5):
    """Extract text and tables from the PDF."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text(layout=True) or ""

    return text

In [18]:
def initialize_model(model_id, device):
    """Initialize or fetch the model and tokenizer."""
    global cached_model, cached_tokenizer
    if not cached_model or not cached_tokenizer:
        cached_tokenizer = AutoTokenizer.from_pretrained(model_id)
        cached_model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if device.type == "mps" else torch.float32,
        ).to(device)
    return cached_tokenizer, cached_model
    

In [19]:
def ask_question(tokenizer, model, context, question):
    """Ask a question to the model."""
    try:
        print(f"Question: {question}")
        input_text = f"{context}\n\nQuestion: {question}"
        inputs = tokenizer.encode(input_text, return_tensors="pt").to(model.device)
        outputs = model.generate(inputs, max_new_tokens=100)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer_start = answer.find("Answer:") + len("Answer:")
        print(f"Answer: {answer[answer_start:].strip()}")
        return answer[answer_start:].strip()
    except Exception as e:
        print(f"Error during question answering: {e}")
        return "Unable to generate an answer."

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(extract_text(pdf_path))


FileNotFoundError: [Errno 2] No such file or directory: './p4.pdf'

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
db = FAISS.from_documents(docs, embeddings)




model = AutoModelForCausalLM.from_pretrained("google/gemma-7b")
tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-7b", padding=True, truncation=True, max_length=512
)


/var/folders/tn/6_4sb_gd0v519hfr8w1k52mm0000gn/T/ipykernel_86006/2175024832.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


NameError: name 'docs' is not defined

In [1]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_tensors='pt',
    max_length=512,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=device
)

NameError: name 'pipeline' is not defined

In [2]:
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

NameError: name 'HuggingFacePipeline' is not defined

In [3]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

NameError: name 'RetrievalQA' is not defined

In [ ]:
qa.invoke("Write an educational story for young children.")
